# Train in AML Compute

Now that we've created an Environment and a Dataset, let's run the training in AML Compute.

To do that, there are several steps: 
1. create a `*.py` file that'll be run by the remote compute
1. Configure a ScriptRun that will execute this script
1. Submit the script for execution

# 1. Create a `*.py` file
There are many ways to create a `*.py` file, but here, we'll just write from Jupyter

In [ ]:
%%writefile ridge-train.py
# Copyright (c) 2021 Microsoft
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT

import click
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from azureml.core import Run
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Get the Run context - this will allow us to log metrics against the run. 
run = Run.get_context()


def read_data(data_path):
    diabetes_df = pd.read_csv(data_path)

    y = diabetes_df.pop('target').values
    X = diabetes_df.values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=0)

    print(
        f"Data contains {len(X_train)} training samples and {len(X_test)} test samples")

    return X_train, X_test, y_train, y_test


def plot_residuals_v_actuals(y, y_hat):
    """Residuals (y-axis) vs. Actuals (x-axis) - colored green"""
    resids = y - y_hat

    fig = plt.figure()
    sns.regplot(x=y, y=resids, color='g')

    plt.title('Residual vs. Actual')
    plt.xlabel("Actual Value")
    plt.ylabel("Residuals")

    plt.close(fig)
    return fig


def plot_predictions(y, y_hat):
    """Predictions (y-axis) vs. Actuals (x-axis)"""
    fig = plt.figure()

    sns.regplot(x=y, y=y_hat, color='b')

    plt.title("Prediction vs. Actual")
    plt.xlabel("Actual Value")
    plt.ylabel("Predicted Value")

    plt.close(fig)
    return fig


def plot_resid_histogram(y, y_hat):
    resids = y - y_hat

    fig = plt.figure()
    sns.histplot(resids, color='g', kde=True)

    plt.title("Residual Histogram")

    plt.close(fig)
    return fig


@click.command()
@click.option("--data-path", type=str, help='The path to the training data', required=True)
@click.option('--alpha', default=0.03, type=float)
def main(data_path: str, alpha: float):

    X_train, X_test, y_train, y_test = read_data(data_path)

    # Create, fit, and test the scikit-learn Ridge regression model
    regression_model = Ridge(alpha=alpha)
    regression_model.fit(X_train, y_train)
    preds = regression_model.predict(X_test)

    # Output the Mean Squared Error to the notebook and to the run
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print(f"RMSE:\t{np.round(rmse,4)}",
          f"MAE:\t{np.round(mae,4)}",
          f"R2:\t{np.round(r2,4)}",
          sep='\n')

    resid_fig = plot_residuals_v_actuals(y_test, preds)
    resid_hist = plot_resid_histogram(y_test, preds)
    pred_plt = plot_predictions(y_test, preds)

    # Log metrics to Azure ML
    # THIS IS THE ONLY AML SPECIFIC CODE HERE #
    run.log('alpha', alpha)
    run.log('model_type', 'Ridge')
    run.log('rmse', rmse)
    run.log('mae', mae)
    run.log('r2', r2)
    run.log_image(name='residuals-v-actuals', plot=resid_fig)
    run.log_image(name='residuals-histogram', plot=resid_hist)
    run.log_image(name='prediction-v-actual', plot=pred_plt)

    # Save the model to the outputs directory for capture
    # Anything saved to ./outputs/ folder will be sent to Azure ML
    # at the end of the run
    joblib.dump(value=regression_model, filename='outputs/model.pkl')


if __name__ == "__main__":
    main()



# 2. Configure a `ScriptRun`

In [ ]:
USER_NAME = 'ENTER_YOUR_NAME_HERE'

ENVIRONMENT_NAME = f'sklearn-{USER_NAME}'
EXPERIMENT_NAME = f'diabetes-{USER_NAME}'
DATASET_NAME = f'diabetes-{USER_NAME}'
COMPUTE_TARGET_NAME = 'cpu-cluster'

DATA_PATH = "./data"
BLOB_PATH = f'/data/{USER_NAME}'

In [ ]:
from azureml.core import Workspace, ScriptRun, ScriptRunConfig, Experiment

ws = Workspace.from_config()
exp = Experiment(ws, EXPERIMENT_NAME)

diabetes_dataset = ws.datasets[DATASET_NAME]
sklearn_env = ws.environments[ENVIRONMENT_NAME]
cpu_cluster = ws.compute_targets[COMPUTE_TARGET_NAME]


In [ ]:
script_run = ScriptRunConfig(
    source_directory = '.',
    script='ridge-train.py',
    environment=sklearn_env,
    compute_target=cpu_cluster,
    arguments=['--data-path', diabetes_dataset.as_mount()]
)

# 3. Submit the run and monitor progress

In [ ]:
run = exp.submit(script_run, tags={"Context": "Remote"})

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

<br><br><br><br><br>






###### Copyright (c) Microsoft Corporation. All rights reserved.  
###### Licensed under the MIT License.